In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 17.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 29.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 24.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 16.3 MB/s eta 0:00:00
  Using cached huggingface_hub-0.15.1-py3-none-any.whl (236 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.4/401.4 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.13.0rc1 requires h5py>=2.9.0, which is not installed.
tensorflow-macos 2.13.0rc1 requires wrapt>=1.11.0, which is not installed.
tensorflow-datasets 4.9.2 requires wrapt, which is not installed.
tensorboard 2.13.0 requires markdow

In [ ]:
pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 MB 25.9 MB/s eta 0:00:0000:0100:01
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch
import numpy as np

/Users/shashwatbindal/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer_for_bert = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [4]:
def bert_question_answer(question, passage, max_len=512):
    # Tokenize input question and passage
    # Add special tokens - [CLS] and [SEP]
    input_ids = tokenizer_for_bert.encode(question, passage, max_length=max_len, truncation=True)

    # Getting number of tokens in 1st sentence (question) and 2nd sentence (passage that contains answer)
    sep_index = input_ids.index(102)
    len_question = sep_index + 1
    len_passage = len(input_ids) - len_question

    # Need to separate question and passage
    # Segment ids will be 0 for question and 1 for passage
    segment_ids = [0] * len_question + [1] * len_passage

    # Converting token ids to tokens
    tokens = tokenizer_for_bert.convert_ids_to_tokens(input_ids)

    # Getting start and end scores for answer
    # Converting input arrays to torch tensors before passing to the model
    start_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))[0]
    end_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))[1]

    # Converting scores tensors to numpy arrays
    start_token_scores = start_token_scores.detach().numpy().flatten()
    end_token_scores = end_token_scores.detach().numpy().flatten()

    # Getting start and end index of answer based on highest scores
    answer_start_index = np.argmax(start_token_scores)
    answer_end_index = np.argmax(end_token_scores)

    # Getting scores for start and end token of the answer
    start_token_score = np.round(start_token_scores[answer_start_index], 2)
    end_token_score = np.round(end_token_scores[answer_end_index], 2)

    # Combining subwords starting with ## and get full words in output.
    # It is because tokenizer breaks words which are not in its vocab.
    answer = tokens[answer_start_index]
    for i in range(answer_start_index + 1, answer_end_index + 1):
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]

    # If the answer didn't find in the passage
    if (answer_start_index == 0) or (start_token_score < 0) or (answer == '[SEP]') or (answer_end_index < answer_start_index):
        answer = "Sorry!, I could not find an answer in the passage."

    return (answer_start_index, answer_end_index, start_token_score, end_token_score, answer)

def split_passage_and_process(question, passage, max_len=10000, chunk_size=500):
    # Split the passage into smaller chunks
    passage_chunks = [passage[i:i+chunk_size] for i in range(0, len(passage), chunk_size)]
    answers = []
    # Process each chunk separately
    for chunk in passage_chunks:
        answer = bert_question_answer(question, chunk, max_len=max_len)
        if answer[-1] != "Sorry!, I could not find an answer in the passage.":
            answers.append(answer[-1])

    # Combine the answers from all chunks
    combined_answer = " ".join(answers)
    return combined_answer

In [5]:
pip install soundfile

Note: you may need to restart the kernel to use updated packages.


In [9]:
import sounddevice as sd
import soundfile as sf

# Set the audio settings
sample_rate = 44100
duration = 5 # Duration in seconds
output_file = "audio.wav"

# Record audio from the laptop's microphone
# print("Recording audio...")
# audio = sd.rec(int(sample_rate * duration), samplerate=sample_rate, channels=1)
# sd.wait()

# # Save the audio to a file
# sf.write(output_file, audio, sample_rate)

# print(f"Audio saved to {output_file}")

In [ ]:
brew install flac

In [ ]:
pip install pyttsx3


In [ ]:
pip install comtypes


In [ ]:
pip install gtts


In [ ]:
pip install python-vlc


In [10]:

import speech_recognition as sr

# Create a recognizer object
# r = sr.Recognizer()

# # Path to the audio file
# audio_file = "audio.wav"

# # Load the audio file
# with sr.AudioFile(audio_file) as source:
#     # Read the audio data
#     audio = r.record(source)

# try:
#     # Recognize speech from the audio
#     text = r.recognize_google(audio)
#     print("Transcription:", text)
# except sr.UnknownValueError:
#     print("Speech recognition could not understand audio")
# except sr.RequestError as e:
#     print("Could not request results from Google Speech Recognition service; {0}".format(e))


In [11]:
def askquestion(audio_file,output_file):
    print("Recording audio...")
    audio = sd.rec(int(sample_rate * duration), samplerate=sample_rate, channels=1)
    sd.wait()

    # Save the audio to a file
    sf.write(output_file, audio, sample_rate)

    print(f"Audio saved to {output_file}")
    r = sr.Recognizer()

    # Path to the audio file

    # Load the audio file
    with sr.AudioFile(audio_file) as source:
        # Read the audio data
        audio = r.record(source)
    text=""    

    try:
        # Recognize speech from the audio
        text = r.recognize_google(audio)
        print("Transcription:", text)
    except sr.UnknownValueError:
        print("Speech recognition could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))

    return text
    

In [13]:
text=askquestion("text.wav","text.wav")
text

Recording audio...
Audio saved to text.wav
Transcription: hello how are you I am good how are you doing


'hello how are you I am good how are you doing'

In [14]:
text

'hello how are you I am good how are you doing'

In [ ]:
pip install gtts

  Using cached gTTS-2.3.2-py3-none-any.whl (28 kB)
Note: you may need to restart the kernel to use updated packages.


In [15]:
from gtts import gTTS

def text_to_speech(text, output_file):
    # Create a gTTS object with the text and desired language
    tts = gTTS(text=text, lang='en')

    # Save the audio to a file
    tts.save(output_file)

# Example usage

output_file = "output.mp3"

text_to_speech(text, output_file)

In [ ]:
pip install pygame

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 29.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pygame

def play_mp3(file_path):
    pygame.mixer.init()
    pygame.mixer.music.load(file_path)
    pygame.mixer.music.play()

pygame 2.4.0 (SDL 2.26.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [17]:
def Passageready():

    output_file2="passage.mp3"
    text_to_speech("Give the passage you want to ask questions from", output_file2)
    play_mp3(output_file2)
    passage=input("Enter the Passage: ")
    return passage


In [18]:
def QuestionAnswer(passage):
    
    output_file="Ask.mp3"
    text_to_speech("Ask the question", output_file)
    play_mp3(output_file)
    question=askquestion("question.wav","question.wav")
    ans  = split_passage_and_process( question, passage)
    output_file3="answer.mp3"
    text_to_speech("The Answer to the question....: "+question+"..."+"is....:"+ans, output_file3)
    play_mp3(output_file3)
    return ans


In [19]:
passage=Passageready()

In [20]:
ans=QuestionAnswer(passage)

Recording audio...
Audio saved to question.wav
Transcription: what is Manoj


In [21]:
ans

'sanoj is bhadwa'